# Geometry Optimisation (T=0K)


In this part of the experiment you will optimise the structure of the MgO crystal you created in the previous section. Since MgO has a fcc structure, the only parameter to optimise is the **a** length. Please notice that the value for **a** is the conventional cell one, but in the generation of the structure, the primitive one is selected (primitive_cell=True). In the output you will find both the primitive and conventional cell lattice parameters. In the next section you will be asked to introduce the optimised lattice parameter. Make sure you select the right one.
<br>

In order to optimise the structure, we must choose a force field or potential model to describe the interactions between the Mg and O ions. GULP use a simple force field for MgO - the Mg ion is given a charge of +2e, the O ion -2e and a simple Buckingham repulsive potential operates between the ions.<br>

**Don't try values higher than 10 (makes no sense and might break the notebook)


In [9]:
#The import first, always.
import numpy as np
import matplotlib.pyplot as plt
from ase.spacegroup import crystal
from ase.calculators.gulp import GULP
from math import cos,pi
import re
import os

#Let's make sure there are no old optimisation files left here by previous run and remove them 
#if they are present
for i in range(1,200):
    if os.path.exists('./mgo_opt_{}.grs'.format(i)):
        os.remove('./mgo_opt_{}.grs'.format(i))
    else:
        break

#label is the name of the calculation
label= 'mgo_opt'
#initial lattice parameter length (this is the conventional cell lattice parameter)
a = 0.1
#creation of the MgO crystal structure
MgO = crystal('MgO',basis=[[0.0,0.0,0.0],[0.5,0.5,0.5]],spacegroup=225,cellpar=[a, a, a, 90, 90, 90],primitive_cell=True)

#some options to let gulp know we want to save the intermediate geometries during the optimisation
options = ['dump every 1 noover {}.grs'.format(label)]
#calc defines the calculator we are going to use and the kind of calcualtion to perform, 
#the first one is a single point energy on the initial geometry.
#Go to http://gulp.curtin.edu.au/gulp/help/help_50_txt.html for a complete list of keywowrds
calc = GULP(label=label,keywords='conp', library='ionic.lib', options=options)
#the following line specify which is the calculator we want to use to get the MgO properties
MgO.calc = calc
#since we are going to save the energy and Mg-O distance of each optimisation step, let's create two empty list to do that
energy = []
MgO_distance = []
#get the energy of the initial geometry (the one given as input) and add that to the energy list we just created
energy.append(round(MgO.get_potential_energy(),6))

#now we are asking for a geometry optimisation at constant pressure ('conp opti')
calc = GULP(label=label,keywords='conp opti', library='ionic.lib', options=options)
#the optimiser used is the gulp internal one
opt = calc.get_optimizer(MgO)
#and the calcualtion is run. The fmax specifies what is the threshold for convergence in units of eV/Angstrom
opt.run(fmax=0.05)

#open the output of the calculation (the file extension for outputs in gulp is .got)
#read the output and save it as the lines list
with open(label+'.got') as o:
    lines = o.readlines()

#initialise an empty list for the lattice parameters    
lattice_param = []

#The following loop reads the output file and looks for the line where the energy
#and the final lattice parameters are reported in order to add them to the lists we created above
for i, line in enumerate(lines):
    m = re.match(r'  Cycle:\s*(\S+)\s*\S+\s*(\S+)',line)
    if m:
        energy.append(float(m.group(2)))
    if line.find('  Final energy =     ') != -1:
        final_energy = line.split()[3]
    if line.find('Final cell parameters and derivatives :') != -1:
        s = i + 2 
        while s <= i+7:
            s += 1
            lattice_param.append(lines[s].split()[1])

energy.pop(0)
energy.append(float(final_energy))
        
print('OPTIMISATION')
#look for the Mg-O distance at each optimisation step from the .grs file
for i in range(len(energy)):
    with open(label+'_{}.grs'.format(i+1)) as o:
        if i ==0 :
            o.seek(0)
            MgO_distance.append(o.readlines()[11].split()[0])
            print('Cycle: {} Mg-O distance = '.format(i),MgO_distance[i],'Energy (eV/cell)', energy[i])
        elif i == len(energy)-1:
            MgO_distance.append(o.readlines()[11].split()[0])
            print('Cycle: {} Mg-O distance = '.format(i),MgO_distance[i],'Energy (eV/cell)', energy[i])
        else:
            o.seek(0)
            MgO_distance.append(o.readlines()[12].split()[0])
            print('Cycle: {} Mg-O distance = '.format(i),MgO_distance[i],'Energy (eV/cell)', energy[i])

#let's add a black line just to make it look tidier
print()
#print the optimised lattice parameters
print('Optimised lattice vectors')
print()
print('Primitive cell')
l = ['a','b','c','alpha','beta','gamma']
for i in range(0,6):
    if i < 3:
        print(l[i],'=',np.round(float(lattice_param[i]),4),'Angstrom',)
    else:
        print(l[i],'=',np.round(float(lattice_param[i]),4),'degrees')
print()
#transform the primitive lattice vectors into the conventional one and print them
print('Conventional cell')
for i in range(0,6):
    if i < 3:
        print(l[i],'=',np.round(float(lattice_param[i])*2.*cos(pi/4),4),'Angstrom',)
    else:
        print(l[i],'=',np.round(float(lattice_param[i])*1.5,4),'degrees')

#Finally, let's remove all the files containing the intermediate geometries of the optimisation.
for i in range(1,200):
    if os.path.exists('./mgo_opt_{}.grs'.format(i)):
        os.remove('./mgo_opt_{}.grs'.format(i))
    else:
        break


OPTIMISATION
Cycle: 0 Mg-O distance =  0.070700 Energy (eV/cell) 100000000.0
Cycle: 1 Mg-O distance =  0.070700 Energy (eV/cell) -41.07531762

Optimised lattice vectors

Primitive cell
a = 0.0707 Angstrom
b = 0.0707 Angstrom
c = 0.0707 Angstrom
alpha = 60.0 degrees
beta = 60.0 degrees
gamma = 60.0 degrees

Conventional cell
a = 0.1 Angstrom
b = 0.1 Angstrom
c = 0.1 Angstrom
alpha = 90.0 degrees
beta = 90.0 degrees
gamma = 90.0 degrees


**Questions:** <br>
1. The optimisation process.    
    - Describe the optimisation process.<br> 
    - Why do we need to optimise the structure? <br>
    - Compare the final structure density with the experimental density.<br>
    - Try and start from a very small value and a very large one for **a** (you need to change the value in the script e.g. 2.2 or 6.2). How is the position of the minimum affected by that?
    - Do larger or smaller values for the starting point require more steps? Why?
